In [1]:
import os, json
import pandas as pd
from IPython.display import GeoJSON
from tools.db_util import get_sql_engine, SQL
SQL_ENGINE = get_sql_engine()
%load_ext sql
%sql $SQL_ENGINE.url

HOME = os.path.expanduser('~')
secrets_path = os.path.join(HOME, 'Documents', 'secrets.json')
with open(secrets_path,'r') as secrets_file:
    MAPBOX = json.load(secrets_file)['mapbox']['token']

In [7]:
with SQL_ENGINE.connect() as conn:
    df = pd.read_sql('SELECT * FROM stations', con=conn)

stations = df #[df.state == 'OR']
GeoJSON(
    data = {
        "type": "MultiPoint", 
        "coordinates": stations[['lon','lat']].values.tolist()
    },
    url_template="https://api.tiles.mapbox.com/v4/{id}/{z}/{x}/{y}.png?access_token="+MAPBOX,
    layer_options={
        "id": "mapbox.satellite",
        "maxZoom": 5, "minZoom": 4
    }
)

<IPython.display.GeoJSON object>

In [8]:
for state in df.state.sort_values().unique():
    print(f"{state}:", (df.state == state).sum())

CA: 709
ID: 6
NV: 41
OR: 9
UT: 6
WA: 1


In [13]:
stations = df[df.state == 'WA']
GeoJSON(
    data = {
        "type": "MultiPoint", 
        "coordinates": stations[['lon','lat']].values.tolist()
    },
    layer_options={
        "maxZoom": 5, "minZoom": 4
    }
)

<IPython.display.GeoJSON object>